In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, LeakyReLU, Dropout
from keras.callbacks import EarlyStopping
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [30]:

# Replace these values with your year and day of the year
def datec(row):
    year = row['year']
    day_of_year = row['day']
    hr = row['hour']
    # Create a datetime object with the year and day_of_year
    date = datetime(int(year), 1, 1, 0) + timedelta(hours=int((day_of_year - 1)*24 + hr))

    return date
def kp_convert(k):
    if(k%10 >= 5) : 
        return float(k/10) + 1
    return float(k/10)

n_input = 24
es = EarlyStopping(monitor='val_loss', baseline = 0.6, patience = 5, mode='min', restore_best_weights=True)
def model_fac():
    model = Sequential()
    model.add(LSTM(256, input_shape=(24, 54), return_sequences=True))
    model.add(LeakyReLU(alpha=0.5))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.35))
    model.add(LSTM(64))
    model.add(Dropout(0.3))
    model.add(Dense(32))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mae')
    return model

In [31]:
df = pd.read_csv('data_final.csv')
train_cols = ['bx', 'by', 'bz', 'row1', 'row2', 'row3', 'row4', 'row5', 
              'row6', 'row7', 'row8', 'row9', 'row10', 'row11', 'row12', 'row13', 'row14', 'row15', 
              'row16', 'row17', 'row18', 'row19', 'row20', 'row21', 'row22', 'row23', 'row24', 'row25', 
              'row26', 'row27', 'row28', 'row29', 'row30', 'row31', 'row32', 'row33', 'row34', 'row35', 
              'row36', 'row37', 'row38', 'row39', 'row40', 'row41', 'row42', 'row43', 'row44', 'row45', 
              'row46', 'row47', 'row48', 'row49', 'row50']
target_cols = ["SW Plasma Temperature, K", 
         "SW Proton Density, N/cm^3", "SW Plasma Speed, km/s","SW Plasma flow long. angle", 
         "SW Plasma flow lat. angle", "Flow pressure", "Kp index", "Dst-index", "AL-index", "AU-index"]


    
train0 = df[train_cols]
scaler = MinMaxScaler()
train0 = pd.DataFrame(scaler.fit_transform(train0), columns = train_cols)


In [ ]:
for tc in target_cols:
    target = df[tc]
    if(tc == 'Kp index'):
        target = df[tc].apply(kp_convert).to_numpy()
    else :
        target = df[tc].to_numpy().reshape(-1, 1)
        target = scaler.fit_transform(target)
        target = target.flatten()

    train = train0.copy()
    train[tc] = target
    train = train.to_numpy()

    k = 8
    m = []
    kf = KFold(n_splits=k)
    for i , (train_indices, test_indices) in enumerate(kf.split(train)):
        train_train_, y_train_ = train[train_indices], target[train_indices]
        train_valid_, y_valid_ = train[test_indices], target[test_indices]

        generator = TimeseriesGenerator(train_train_, y_train_, length=n_input, batch_size=32)
        v_generator = TimeseriesGenerator(train_valid_, y_valid_, length=n_input, batch_size=32)
        model = model_fac()
        history = model.fit(generator,epochs=10, validation_data=v_generator, shuffle=False, callbacks=[es], verbose=1)
        m.append(model)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model Loss Progress')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(['Train', 'Validation'], loc='upper right')
        plt.show()

    averaged_model = model_fac()  # Create a new model with the same architecture

    for j in range(k):
        # Load the trained model for this fold
        model = m[j]

        # Iterate through the layers and average the weights and biases
        for layer_avg, layer in zip(averaged_model.layers, model.layers):
            if layer_avg.get_weights():
                w = []
                for l, r in zip(layer_avg.get_weights(), layer.get_weights()):
                    w.append((l+r)/2)
                layer_avg.set_weights(w)

    averaged_model.save(f'/kaggle/working/model_{tc}.h5')
